In [37]:
from folium import plugins , GeoJson
from folium import Icon, Element, CircleMarker, Map, Popup, LayerControl

import pandas as pd

In [11]:
df = pd.read_csv('../Data/modified_Thawra_2019-2020 - Main.csv', header=1, parse_dates=['Startdate'])

In [12]:
df = df[['Startdate','gpslatend', 'gpslongend','demandtypeone']].sample(100)
df

,Startdate,gpslatend,gpslongend,demandtypeone
2185,2019-12-16,34.532032,36.060027,Thawra 2019
305,2019-10-20,34.350752,35.717377,Thawra 2019
1563,2019-11-22,34.542409,36.077015,Corruption
1755,2019-11-29,33.624634,35.494341,Thawra 2019
1760,2019-11-29,34.572991,36.281179,Social
...,...,...,...,...
1460,2019-11-18,33.877299,35.517442,Labor
3009,2020-02-11,33.900537,35.495969,Thawra 2019
958,2019-11-07,33.896038,35.493737,Thawra 2019
1392,2019-11-15,33.833264,35.913899,Thawra 2019


In [13]:
df = df.reset_index().drop(columns='index').dropna()
df

,Startdate,gpslatend,gpslongend,demandtypeone
0,2019-12-16,34.532032,36.060027,Thawra 2019
1,2019-10-20,34.350752,35.717377,Thawra 2019
2,2019-11-22,34.542409,36.077015,Corruption
3,2019-11-29,33.624634,35.494341,Thawra 2019
4,2019-11-29,34.572991,36.281179,Social
...,...,...,...,...
95,2019-11-18,33.877299,35.517442,Labor
96,2020-02-11,33.900537,35.495969,Thawra 2019
97,2019-11-07,33.896038,35.493737,Thawra 2019
98,2019-11-15,33.833264,35.913899,Thawra 2019


In [29]:
def color_demand(demand):
    d = {'Corruption':'red', 'Politics (local)':'blue', 'Thawra 2019':'green',
       'Economic/Financial':'purple', 'Human rights / civic freedoms':'orange', 'Social':'darkred',
       'Politics (national)':'lightred', "Women's rights":'pink',
       'Politics (regional/international)':'darkblue', 'Religion/Sectarian':'red',
       'Security':'darkgreen', 'Labor':'gray'}
    color= d[demand]
    return color
#df['color'] = df['demandtypeone'].apply(color_demand)

In [31]:
color_demand('Politics (regional/international)')

'darkblue'

In [16]:
df

,Startdate,gpslatend,gpslongend,demandtypeone,color
0,2019-12-16,34.532032,36.060027,Thawra 2019,green
1,2019-10-20,34.350752,35.717377,Thawra 2019,green
2,2019-11-22,34.542409,36.077015,Corruption,red
3,2019-11-29,33.624634,35.494341,Thawra 2019,green
4,2019-11-29,34.572991,36.281179,Social,darkred
...,...,...,...,...,...
95,2019-11-18,33.877299,35.517442,Labor,gray
96,2020-02-11,33.900537,35.495969,Thawra 2019,green
97,2019-11-07,33.896038,35.493737,Thawra 2019,green
98,2019-11-15,33.833264,35.913899,Thawra 2019,green


In [50]:
# Create a map object
m = Map(location=[33.905074, 35.744929])

# Create the data for the TimestampedGeoJson plugin
data = {
    'type': 'FeatureCollection',
    'features': []
}

for i, row in df.iterrows():
    t, lat, lon, demand, color = row['Startdate'], row['gpslatend'], row['gpslongend'], row['demandtypeone'], row['color']
    # CircleMarker(location=[lat, lon],
    #              radius=5,
    #              color=color,
    #              popup=demand,
    #              fill_color=color,
    #              fill=True).add_to(m)
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [lon, lat],
        },
        'properties': {
            'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
            'value':demand
        }
    }
    data['features'].append(feature)

# Add a timeline to the map
timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=False).add_to(m)

# # Add a timeline to the map

# coordinates = []
# time = []
# for indx in df.index:
#     coordinates.append([df['gpslatend'][indx], df['gpslongend'][indx]])
#     time.append(df['Startdate'][indx].strftime('%Y-%m-%dT%H:%M:%S'))

# timeline = plugins.TimestampedGeoJson({
#     'type': 'FeatureCollection',
#     'features': [
#         {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': coordinates,
#             },
#             'properties': {
#                 'time': time
#             }
#         }
#     ]
# }, period='PT1M', add_last_point=True).add_to(m)

# Display the map
m

In [137]:
# layers = {}
# for value in df['value'].unique():
#     # Create the data for the TimestampedGeoJson plugin
#     data = {
#         'type': 'FeatureCollection',
#         'features': []
#     }
#     for i, row in df[df['value'] == value].iterrows():
#         lon, lat, t, value = row['lon'], row['lat'], row['time'], row['value']
#         feature = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': [lon, lat],
#             },
#             'properties': {
#                 'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
#                 'value': value
#             }
#         }
#         data['features'].append(feature)


In [51]:
# Create a map object

m2 = Map(location=[33.905074, 35.744929])

# Create a list of GeoJson objects, one for each value

geojson_list = []

for demand in df['demandtypeone'].unique():
    # Create the data for the GeoJson object
    data = {
        'type': 'FeatureCollection',
        'features': []
    }
    for i, row in df[df['demandtypeone'] == demand].iterrows():
        lon, lat, t, color = row['gpslongend'], row['gpslatend'], row['Startdate'], row['color']
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',  # Set the type to something other than 'point'
                'coordinates': [lon, lat],  # Set the coordinates of the feature
            },
            'properties': {
                'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
                'color': color,
                'value':demand
            }
        }
        # Add a popup to the feature
        popup = Popup(demand)
        feature['properties']['popup'] = str(popup)
        data['features'].append(feature)



    # Create a GeoJson object and add it to the map
    def style_function(feature):
        color = feature['properties']['color']
        return {'color': color}

    geojson = GeoJson(data, style_function=style_function).add_to(m2)
   
    geojson_list.append(geojson)

# LayerControl(position='topright', collapsed=False).add_to(m2)

timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=False).add_to(m2)

# Display the map
m2